In [17]:
import pandas as pd
import numpy as np

In [18]:
application_record = pd.read_csv('application_record.csv', index_col=0)
application_record.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
ID,,,,,,,,,,,,,,,,,
5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [19]:
credit_record = pd.read_csv('credit_record.csv')
credit_record.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


# Data Cleaning

<br> As evident from the credit_record table, the ID is not unique. Ideally, we aim to group the dataframe by ID and extract all status records for each person. This objective can be accomplished through the pivot table function. <br><br>

In [20]:
# Pivoting the original DataFrame to get status counts as columns
pivoted_df = credit_record.pivot_table(index='ID', columns='STATUS', aggfunc='size', fill_value=0).reset_index()

pivoted_df.head()

STATUS,ID,0,1,2,3,4,5,C,X
0,5001711,3,0,0,0,0,0,0,1
1,5001712,10,0,0,0,0,0,9,0
2,5001713,0,0,0,0,0,0,0,22
3,5001714,0,0,0,0,0,0,0,15
4,5001715,0,0,0,0,0,0,0,60


<br> We conduct an inner join between the application record and our newly created pivot table. Our focus lies in predicting customers' credit behavior, thus we solely prioritize IDs that are present in both tables, ensuring we have both predictors and outcomes available. <br><br>

In [21]:
merged_df = pd.merge(application_record, pivoted_df, how="inner", on="ID")

merged_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,OCCUPATION_TYPE,CNT_FAM_MEMBERS,0,1,2,3,4,5,C,X
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,NaN,2.0,1,1,0,0,0,0,13,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,NaN,2.0,1,1,0,0,0,0,12,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,Security staff,2.0,7,0,0,0,0,0,7,16
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,Sales staff,1.0,2,0,0,0,0,0,0,3
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,Sales staff,1.0,0,0,0,0,0,0,0,5


In [22]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   36457 non-null  int64  
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   CNT_CHILDREN         36457 non-null  int64  
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_BIRTH           36457 non-null  int64  
 11  DAYS_EMPLOYED        36457 non-null  int64  
 12  FLAG_MOBIL           36457 non-null  int64  
 13  FLAG_WORK_PHONE      36457 non-null  int64  
 14  FLAG_PHONE           36457 non-null  int64  
 15  FLAG_EMAIL           36457 non-null 

In [23]:
# Converting column types to Int 64
columns = merged_df.iloc[:, 17:].columns
merged_df[columns] = merged_df[columns].astype('Int64')

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   36457 non-null  int64  
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   CNT_CHILDREN         36457 non-null  int64  
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_BIRTH           36457 non-null  int64  
 11  DAYS_EMPLOYED        36457 non-null  int64  
 12  FLAG_MOBIL           36457 non-null  int64  
 13  FLAG_WORK_PHONE      36457 non-null  int64  
 14  FLAG_PHONE           36457 non-null  int64  
 15  FLAG_EMAIL           36457 non-null 

# Brief overview on the numerical features
merged_df.describe()

In [25]:
merged_df['DAYS_EMPLOYED'].sort_values(ascending = False).unique()

array([365243,    -17,    -43, ..., -15227, -15661, -15713], dtype=int64)

<br> In this dataset, the days_employed values are negative, with the exception of the positive value (365243), which indicates unemployment. To simplify comprehension, all values have been converted to positive, where 0 now signifies unemployment. This conversion has also been applied to the days_birth variable for consistency. <br><br>

In [26]:
# Positive values
merged_df['DAYS_EMPLOYED'] = merged_df['DAYS_EMPLOYED'].apply(lambda x : 0 if x > 0 else -x)

merged_df['DAYS_BIRTH'] = merged_df['DAYS_BIRTH'] * -1

# Renaming columns
merged_df.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Property','CNT_CHILDREN':'Children','AMT_INCOME_TOTAL':'Income',
'NAME_INCOME_TYPE':'Income status', 'NAME_EDUCATION_TYPE':'Education','NAME_FAMILY_STATUS':'Marital', 'NAME_HOUSING_TYPE':'Housing', 'DAYS_BIRTH': 'Day since birth',
'DAYS_EMPLOYED' : 'Days employed', 'FLAG_MOBIL' : 'Mobile', 'FLAG_WORK_PHONE':'Work phone', 'FLAG_PHONE':'Phone', 'FLAG_EMAIL':'Email', 'OCCUPATION_TYPE':'Job',
'CNT_FAM_MEMBERS':'Family'},inplace=True)

merged_df.head()

,ID,Gender,Car,Property,Children,Income,Income status,Education,Marital,Housing,...,Job,Family,0,1,2,3,4,5,C,X
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,NaN,2,1,1,0,0,0,0,13,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,...,NaN,2,1,1,0,0,0,0,12,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,...,Security staff,2,7,0,0,0,0,0,7,16
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,Sales staff,1,2,0,0,0,0,0,0,3
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,...,Sales staff,1,0,0,0,0,0,0,0,5


In [27]:
# Saving the combined dataframe to prevent the need for rerunning it.
merged_df.to_csv('merged_df.csv', index = False)